In [0]:
%sql
DROP TABLE IF EXISTS fmcg.bronze.customers;
DROP TABLE IF EXISTS fmcg.bronze.products;
DROP TABLE IF EXISTS fmcg.bronze.gross_price;

In [0]:
from pyspark.sql import functions as F
from delta.tables import DeltaTable

In [0]:
s3_path_customer = "s3://sportsbar-dp-dbx/customers/*.csv"
s3_path_gross_price = "s3://sportsbar-dp-dbx/gross_price/*.csv"
s3_path_products = "s3://sportsbar-dp-dbx/products/*.csv"

In [0]:
def store_into_bronze(list_s3):
    for s3_path in list_s3:
        # reading the file from s3
        df = (
        spark.read
        .option("header", "true")
        .option("inferSchema", "true")
        .option("includeMetadata", "true")
        .csv(s3_path[0])
        )
        #adding the extra columns
        df = (
            df
            .withColumn("read_timestamp", F.current_timestamp())
            .withColumn("_metadata_filename", F.col("_metadata.file_path"))
            .withColumn("_metadata_file_size", F.col("_metadata.file_size"))
        )
        #writting it to bronze layer
        (df.write
        .format("delta")
        .option("delta.enableChangeDataFeed", "true")
        .mode("overwrite") 
        .saveAsTable(f"fmcg.bronze.{s3_path[1]}")
        )
    return (print("stored tables into bronze layer successfully!!"))

In [0]:
list_s3 = [[s3_path_customer,"customers"], [s3_path_gross_price,"gross_price"], [s3_path_products,"products"]]
store_into_bronze(list_s3)

In [0]:
df = spark.sql("select * from fmcg.bronze.customers")
display(df)